**Executed:** Tue Feb 21 16:16:40 2017

**Duration:** 4 seconds.

In [1]:
# default values of file paths, assuming quantum_fog is working dir
in_bif = 'examples_cbnets/earthquake.bif'
in_dot = 'examples_cbnets/earthquake.dot'
in_csv = 'learning/training_data_c/earthquake.csv'

In [2]:
# Cell inserted during automated execution.
in_bif = 'examples_cbnets/asia_.bif'
in_dot = 'examples_cbnets/asia_.dot'
in_csv = 'learning/training_data_c/tempo.csv'

**Parameter Learning Template**

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Learned-Network-Parameters" data-toc-modified-id="Learned-Network-Parameters-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Learned Network Parameters</a></div><div class="lev2 toc-item"><a href="#QFog" data-toc-modified-id="QFog-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>QFog</a></div><div class="lev2 toc-item"><a href="#bnlearn" data-toc-modified-id="bnlearn-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>bnlearn</a></div><div class="lev2 toc-item"><a href="#Summary-of-Running-Times" data-toc-modified-id="Summary-of-Running-Times-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Summary of Running Times</a></div>

In [3]:
import pandas as pd
import numpy as np
from graphviz import Source

import warnings
warnings.filterwarnings("ignore", module="rpy2")

import rpy2
%load_ext rpy2.ipython
%R library("bnlearn");
%R library("Rgraphviz");

In [4]:
# make this the absolute path of quantum-fog folder
qfog_path = '/home/jupyter/Notebooks/quantum-fog'

In [5]:
import os
import sys
if qfog_path != os.getcwd():
    os.chdir('../../')
    sys.path.insert(0,os.getcwd())
from learning.NetParamsLner import *

Read in_csv, create Pandas Dataframe with it, push dataframe into R

In [6]:
states_df = pd.read_csv(in_csv)
states_df.head()

,Smoke,Bronchitis,LungCancer,Visited_AsianTBHRA,Tuberculosis,Either,Dyspnea,XRay
0,0,0,1,1,1,1,0,1
1,1,0,1,1,1,1,0,1
2,1,0,1,1,1,1,0,1
3,0,0,1,1,1,1,0,1
4,0,0,1,1,1,1,0,1


In [7]:
states_df.tail()

,Smoke,Bronchitis,LungCancer,Visited_AsianTBHRA,Tuberculosis,Either,Dyspnea,XRay
1995,0,0,1,1,1,1,0,1
1996,0,1,1,1,1,1,1,1
1997,0,0,1,1,1,1,0,1
1998,1,1,1,1,1,1,1,1
1999,1,0,1,1,1,1,0,1


In [8]:
%Rpush states_df
%R str(states_df)

'data.frame':	2000 obs. of  8 variables:
 $ Smoke             : int  0 1 1 0 0 0 0 0 1 0 ...
 $ Bronchitis        : int  0 0 0 0 0 0 0 0 1 0 ...
 $ LungCancer        : int  1 1 1 1 1 1 1 1 1 1 ...
 $ Visited_AsianTBHRA: int  1 1 1 1 1 1 1 1 1 1 ...
 $ Tuberculosis      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ Either            : int  1 1 1 1 1 1 1 1 1 1 ...
 $ Dyspnea           : int  0 0 0 0 0 0 0 0 1 0 ...
 $ XRay              : int  1 1 1 1 1 1 1 1 1 1 ...


For bnlearn, data.frame columns cannot be int type, must be changed to factor.
data.frame is a list so can use lapply

In [9]:
%R states_df[] <- lapply(states_df, factor)
%R str(states_df)

'data.frame':	2000 obs. of  8 variables:
 $ Smoke             : Factor w/ 2 levels "0","1": 1 2 2 1 1 1 1 1 2 1 ...
 $ Bronchitis        : Factor w/ 2 levels "0","1": 1 1 1 1 1 1 1 1 2 1 ...
 $ LungCancer        : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ Visited_AsianTBHRA: Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ Tuberculosis      : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ Either            : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ Dyspnea           : Factor w/ 2 levels "0","1": 1 1 1 1 1 1 1 1 2 1 ...
 $ XRay              : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...


# Learned Network Parameters

## QFog

In [10]:
# emp = empirical, learned
is_quantum = False
bnet = BayesNet.read_bif(in_bif, is_quantum)
bnet_emp = BayesNet.read_dot(in_dot)
vtx_to_states = bnet.get_vtx_to_state_names()
bnet_emp.import_nd_state_names(vtx_to_states)


lnr = NetParamsLner(is_quantum, bnet_emp, states_df)

In [11]:
%%capture qfog_params_time
%time lnr.learn_all_bnet_pots()

In [12]:
print(qfog_params_time)

CPU times: user 51.7 ms, sys: 3.98 ms, total: 55.7 ms
Wall time: 55.9 ms



In [13]:
lnr.compare_true_and_emp_pots(bnet, bnet_emp)


node= Bronchitis
true:
['Smoke', 'Bronchitis']
[[ 0.6  0.4]
 [ 0.3  0.7]]
empirical:
['Smoke', 'Bronchitis']
[[ 0.62461851  0.37538149]
 [ 0.31366765  0.68633235]]

node= Either
true:
['LungCancer', 'Tuberculosis', 'Either']
[[[ 1.  0.]
  [ 1.  0.]]

 [[ 1.  0.]
  [ 0.  1.]]]
empirical:
['LungCancer', 'Tuberculosis', 'Either']
[[[ 1.  0.]
  [ 1.  0.]]

 [[ 1.  0.]
  [ 0.  1.]]]

node= Smoke
true:
['Smoke']
[ 0.5  0.5]
empirical:
['Smoke']
[ 0.4915  0.5085]

node= Visited_AsianTBHRA
true:
['Visited_AsianTBHRA']
[ 0.01  0.99]
empirical:
['Visited_AsianTBHRA']
[ 0.004  0.996]

node= LungCancer
true:
['Smoke', 'LungCancer']
[[ 0.1   0.9 ]
 [ 0.01  0.99]]
empirical:
['Smoke', 'LungCancer']
[[ 0.09664293  0.90335707]
 [ 0.00983284  0.99016716]]

node= Tuberculosis
true:
['Visited_AsianTBHRA', 'Tuberculosis']
[[ 0.05  0.95]
 [ 0.01  0.99]]
empirical:
['Visited_AsianTBHRA', 'Tuberculosis']
[[ 0.          1.        ]
 [ 0.00953815  0.99046185]]

node= Dyspnea
true:
['Bronchitis', 'Either', 'Dy

## bnlearn


In [14]:
%Rpush in_bif
%R bn.fit = read.bif(in_bif)
%R str(bn.fit)

List of 8
 $ Visited_AsianTBHRA:List of 4
  ..$ node    : chr "Visited_AsianTBHRA"
  ..$ parents : chr(0) 
  ..$ children: chr "Tuberculosis"
  ..$ prob    : table [1:2(1d)] 0.01 0.99
  .. ..- attr(*, "dimnames")=List of 1
  .. .. ..$ : chr [1:2] "yes" "no"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ Tuberculosis      :List of 4
  ..$ node    : chr "Tuberculosis"
  ..$ parents : chr "Visited_AsianTBHRA"
  ..$ children: chr "Either"
  ..$ prob    : table [1:2, 1:2] 0.05 0.95 0.01 0.99
  .. ..- attr(*, "dimnames")=List of 2
  .. .. ..$ Tuberculosis      : chr [1:2] "yes" "no"
  .. .. ..$ Visited_AsianTBHRA: chr [1:2] "yes" "no"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ Smoke             :List of 4
  ..$ node    : chr "Smoke"
  ..$ parents : chr(0) 
  ..$ children: chr [1:2] "LungCancer" "Bronchitis"
  ..$ prob    : table [1:2(1d)] 0.5 0.5
  .. ..- attr(*, "dimnames")=List of 1
  .. .. ..$ : chr [1:2] "yes" "no"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ LungCancer        :List o

In [15]:
%R bn = bn.net(bn.fit)
%R str(bn)

List of 3
 $ learning:List of 6
  ..$ whitelist: NULL
  ..$ blacklist: NULL
  ..$ test     : chr "none"
  ..$ ntests   : num 0
  ..$ algo     : chr "empty"
  ..$ args     : list()
 $ nodes   :List of 8
  ..$ Visited_AsianTBHRA:List of 4
  .. ..$ mb      : chr "Tuberculosis"
  .. ..$ nbr     : chr "Tuberculosis"
  .. ..$ parents : chr(0) 
  .. ..$ children: chr "Tuberculosis"
  ..$ Tuberculosis      :List of 4
  .. ..$ mb      : chr [1:3] "Visited_AsianTBHRA" "LungCancer" "Either"
  .. ..$ nbr     : chr [1:2] "Visited_AsianTBHRA" "Either"
  .. ..$ parents : chr "Visited_AsianTBHRA"
  .. ..$ children: chr "Either"
  ..$ Smoke             :List of 4
  .. ..$ mb      : chr [1:2] "LungCancer" "Bronchitis"
  .. ..$ nbr     : chr [1:2] "LungCancer" "Bronchitis"
  .. ..$ parents : chr(0) 
  .. ..$ children: chr [1:2] "LungCancer" "Bronchitis"
  ..$ LungCancer        :List of 4
  .. ..$ mb      : chr [1:3] "Tuberculosis" "Smoke" "Either"
  .. ..$ nbr     : chr [1:2] "Smoke" "Either"
  .. ..$ pa

In [16]:
%%capture bnlearn_params_time
%time %R rfit = bn.fit(bn, data = states_df);

In [17]:
print(bnlearn_params_time)

CPU times: user 20.6 ms, sys: 37 us, total: 20.6 ms
Wall time: 21 ms



In [18]:
%R str(rfit)

List of 8
 $ Visited_AsianTBHRA:List of 4
  ..$ node    : chr "Visited_AsianTBHRA"
  ..$ parents : chr(0) 
  ..$ children: chr "Tuberculosis"
  ..$ prob    : table [1:2(1d)] 0.004 0.996
  .. ..- attr(*, "dimnames")=List of 1
  .. .. ..$ : chr [1:2] "0" "1"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ Tuberculosis      :List of 4
  ..$ node    : chr "Tuberculosis"
  ..$ parents : chr "Visited_AsianTBHRA"
  ..$ children: chr "Either"
  ..$ prob    : table [1:2, 1:2] 0 1 0.00954 0.99046
  .. ..- attr(*, "dimnames")=List of 2
  .. .. ..$ Tuberculosis      : chr [1:2] "0" "1"
  .. .. ..$ Visited_AsianTBHRA: chr [1:2] "0" "1"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ Smoke             :List of 4
  ..$ node    : chr "Smoke"
  ..$ parents : chr(0) 
  ..$ children: chr [1:2] "LungCancer" "Bronchitis"
  ..$ prob    : table [1:2(1d)] 0.491 0.508
  .. ..- attr(*, "dimnames")=List of 1
  .. .. ..$ : chr [1:2] "0" "1"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ LungCancer        :List of 4
  

In [19]:
%%R
j = 0
for( nd in nodes(bn.fit)){
    j = j + 1
    cat('----------------', nd[1], '\n')
    cat('true:\n')
    print(coef(bn.fit)[j])
    cat('empirical:\n')
    print(coef(rfit)[j])
}

---------------- Visited_AsianTBHRA 
true:
$Visited_AsianTBHRA
 yes   no 
0.01 0.99 

empirical:
$Visited_AsianTBHRA
    0     1 
0.004 0.996 

---------------- Tuberculosis 
true:
$Tuberculosis
            Visited_AsianTBHRA
Tuberculosis  yes   no
         yes 0.05 0.01
         no  0.95 0.99

empirical:
$Tuberculosis
            Visited_AsianTBHRA
Tuberculosis           0           1
           0 0.000000000 0.009538153
           1 1.000000000 0.990461847

---------------- Smoke 
true:
$Smoke
yes  no 
0.5 0.5 

empirical:
$Smoke
     0      1 
0.4915 0.5085 

---------------- LungCancer 
true:
$LungCancer
          Smoke
LungCancer  yes   no
       yes 0.10 0.01
       no  0.90 0.99

empirical:
$LungCancer
          Smoke
LungCancer           0           1
         0 0.096642930 0.009832842
         1 0.903357070 0.990167158

---------------- Bronchitis 
true:
$Bronchitis
          Smoke
Bronchitis yes  no
       yes 0.6 0.3
       no  0.4 0.7

empirical:
$Bronchitis
          Smoke

## Summary of Running Times

In [20]:
print("QFog:\n", qfog_params_time)
print("bnlearn:\n", bnlearn_params_time)

QFog:
 CPU times: user 51.7 ms, sys: 3.98 ms, total: 55.7 ms
Wall time: 55.9 ms

bnlearn:
 CPU times: user 20.6 ms, sys: 37 us, total: 20.6 ms
Wall time: 21 ms

